https://towardsdatascience.com/blockchain-explained-in-7-python-functions-c49c84f34ba5

In [28]:
import hashlib
import json

In [29]:
def hash_function(k):
    if type(k) is not str:
        k = json.dumps(k, sort_keys=True)
    return hashlib.sha256(k.encode("utf8")).hexdigest()  ## had to add the encode("utf8")

In [13]:
hash_function("i'm testing")

'13b5c13435b444cc1375e890ac618b9b69ccfd2b2e993dcf67e4f25065e7379b'

In [30]:
def update_state(transaction, state):
    state = state.copy()
    
    for key in transaction:
        if key in state.keys():
            state[key] += transaction[key]
        else:
            state[key] = transaction[key]
            
    return state

In [17]:
t = {'transaction': {'Tom': 9, 'Medium': 1}}
s = {'state': {'Tom': 8, 'Medium': 2}}

In [18]:
print(s)
update_state(t,s)

{'state': {'Tom': 8, 'Medium': 2}}


{'state': {'Tom': 8, 'Medium': 2}, 'transaction': {'Tom': 9, 'Medium': 1}}

In [19]:
t = {'transaction': {'Tom': 6, 'Medium': 4}}

In [20]:
update_state(t,s)

{'state': {'Tom': 8, 'Medium': 2}, 'transaction': {'Tom': 6, 'Medium': 4}}

In [31]:
def valid_transaction(transaction,state):
    """a valid transaction must sum to zero."""
    if sum(transaction.values()) is not 0:
        return False
    for key in transaction.keys():
        if key in state.keys():
            account_balance = state[key]
        else:
            account_balance = 0
            
        if account_balance + transaction[key] < 0:
            return False
    return True

In [32]:
def make_block(transactions, chain):
    """make a block to go on the chain"""
    parent_hash = chain[-1]['hash']
    block_number = chain[-1]['contents']['block_number'] + 1
    
    block_contents = {
        'block_number': block_number,
        'parent_hash': parent_hash,
        'transaction_count': block_number + 1,
        'transaction':transactions
    }
    
    return {'hash': hash_function(block_contents), 'contents': block_contents}

In [33]:
def check_block_hash(block):
    expected_hash = hash_function(block['contents'])
    if block['hash'] is not expected_hash:
        raise
    return

In [34]:
def check_chain(chain):
    """check that chain is valid"""
    if type(chain) is str:
        try:
            chain = json.loads(chain)
            assert (type(chain) == list)
        except ValueError:
            #string passed was not valid JSON
            return False
    elif type(chain) is not list:
        return False
    
    state = {}
    
    for transaction in chain[0]['contents']['transaction']:
        state = update_state(transaction, state)
        
    check_block_hash(chain[0])
    parent = chain[0]
    
    for block in chain[1:]:
        state = check_block_validity(block, parent, state)
        parent = block
        
    return state

In [35]:
def add_transaction_to_chain(transaction, state, chain):
    if valid_transaction(transaction, state):
        state = update_state(transaction, state)
    else: 
        raise Exception('Invalid transaction')
    
    my_block = make_block(state, chain)
    chain.append(my_block)
    
    for transaction in chain:
        check_chain(transaction)
        
    return state, chain

In [36]:
genesis_block = {
    'hash': hash_function({
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 10}]
    }),
    'contents': {
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 10}]
    },
}

In [37]:
block_chain = [genesis_block]
chain_state = {'Tom': 10}

In [38]:
chain_state, block_chain = add_transaction_to_chain(
    transaction={'Tom': -1, 'Medium': 1}, state = chain_state, chain = block_chain
)

In [39]:
chain_state

{'Tom': 9, 'Medium': 1}

In [40]:
block_chain

[{'hash': '064d0b480b3b92761f31831d30ae9f01954efaa62371b4b44f11465ec22abe93',
  'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction_count': 1,
   'transaction': [{'Tom': 10}]}},
 {'hash': 'b4ae25f0cc0ee0b0caa66b9a3473e9a108652d53b1dc22a40962fef5c8c0f08c',
  'contents': {'block_number': 1,
   'parent_hash': '064d0b480b3b92761f31831d30ae9f01954efaa62371b4b44f11465ec22abe93',
   'transaction_count': 2,
   'transaction': {'Tom': 9, 'Medium': 1}}}]

In [41]:
chain_state, block_chain = add_transaction_to_chain(
    transaction={'Tom': -1, 'Fred': 1}, state = chain_state, chain = block_chain
)

In [42]:
chain_state

{'Tom': 8, 'Medium': 1, 'Fred': 1}

In [43]:
block_chain

[{'hash': '064d0b480b3b92761f31831d30ae9f01954efaa62371b4b44f11465ec22abe93',
  'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction_count': 1,
   'transaction': [{'Tom': 10}]}},
 {'hash': 'b4ae25f0cc0ee0b0caa66b9a3473e9a108652d53b1dc22a40962fef5c8c0f08c',
  'contents': {'block_number': 1,
   'parent_hash': '064d0b480b3b92761f31831d30ae9f01954efaa62371b4b44f11465ec22abe93',
   'transaction_count': 2,
   'transaction': {'Tom': 9, 'Medium': 1}}},
 {'hash': '3f90180b88818649f3cb6250fde5016f0b8993aab1fdd13ebff6974694e12bfe',
  'contents': {'block_number': 2,
   'parent_hash': 'b4ae25f0cc0ee0b0caa66b9a3473e9a108652d53b1dc22a40962fef5c8c0f08c',
   'transaction_count': 3,
   'transaction': {'Tom': 8, 'Medium': 1, 'Fred': 1}}}]

In [44]:
chain_state, block_chain = add_transaction_to_chain(
    transaction={'Tom': -4, 'James': 4}, state = chain_state, chain = block_chain
)

In [45]:
chain_state, block_chain = add_transaction_to_chain(
    transaction={'Sally': 2, 'James': -2}, state = chain_state, chain = block_chain
)

In [46]:
chain_state

{'Tom': 4, 'Medium': 1, 'Fred': 1, 'James': 2, 'Sally': 2}

In [47]:
### start a new block chain
genesis_block2 = {
    'hash': hash_function({
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 1000}]
    }),
    'contents': {
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 1000}]
    },
}

In [51]:
block_chain2 = [genesis_block2]
chain_state2 = {'Tom': 1000}

In [52]:
block_chain2

[{'hash': 'f7b86a0146f670853cf0cff557ddeef6b76ba496dec97a30103560224f77cf2d',
  'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction_count': 1,
   'transaction': [{'Tom': 1000}]}}]

In [53]:
chain_state2, block_chain2 = add_transaction_to_chain(
    transaction={'Tom': -200, 'James': 200}, state = chain_state2, chain = block_chain2
)

In [54]:
chain_state2

{'Tom': 800, 'James': 200}

### Hashing more indepth  
https://www.pythoncentral.io/hashing-strings-with-python/

In [3]:
import hashlib
hash_object = hashlib.sha256(b'Hello worlds')
hash_dig = hash_object.hexdigest()
print(hash_dig)

a26c244a30b716788c8d1c3720bbd976cb70e309c822a57f45a67927def06052


In [11]:
## checking passwords
import uuid
def hash_password(password):
    salt = uuid.uuid4().hex
    print(salt)
    return hashlib.sha256(salt.encode() + password.encode()).hexdigest() + ':' + salt

In [13]:
hash_password('george')

c7ee6d1755fe4643b29164c682137aec


'cdcdff822eac4297b4fce8bf50af71bceeb429790cdda2fa11f33cc42aa71708:c7ee6d1755fe4643b29164c682137aec'

In [39]:
def check_password(hashed_password, new_password):
    password, salt = hashed_password.split(':')
    return password == hashlib.sha256(salt.encode() + new_password.encode()).hexdigest()

In [34]:
old_pass = input('please input your password: ',)

please input your password: qwerty123


In [35]:
new_pass = input('please repeat your password: ',)

please repeat your password: qwerty123


In [40]:
print(old_pass, new_pass)
old_pass_hash = hash_password(old_pass)
print(old_pass_hash)
print(hash_password(new_pass))
check_password(old_pass_hash, new_pass)

qwerty123 qwerty123
8285b40975fe466f84d5a446f6043974
ce4ac8017c7eb32498231f6a63d152ea42b350acdbc2c65682d6b6d0db8d8524:8285b40975fe466f84d5a446f6043974
442ae44fcf74472e8e96dfa9e6cefe31
00149f51ddde13cbd50ee4444ba93d6cd0e557f82349215bbf777aa9c064066d:442ae44fcf74472e8e96dfa9e6cefe31


True